<a href="https://colab.research.google.com/github/LaJeremi/Tensorflow-nlp-tutorial-Practice-/blob/main/09.%20Word%20Embedding/%2009_07_%EC%9E%90%EB%AA%A8_%EB%8B%A8%EC%9C%84_%ED%95%9C%EA%B5%AD%EC%96%B4_FastText_%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **09-07 자모 단위 한국어 FastText 학습하기**

출처: https://wikidocs.net/121755

출처: https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/09.%20Word%20Embedding/9-7.%20jamo_fasttext.ipynb

In [ ]:

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 9.43 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 33.4 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-02-15 09:30:41--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::

In [ ]:

# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=b7e2ee5da6036926706b3e7c06033588338058a42f39ea321d16780af966fcee
  Stored in directory: /root/.cache/pip/wheels/93/33/b8/bc2256172a415340e34f3c11ef2b0f3f391769000bb74de988
Successfully built hgtk


In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (943/943), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 3930 (delta 854), reused 806 (delta 806), pack-reused 2987
Receiving objects: 100% (3930/3930), 8.24 MiB | 27.85 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
/content/Mecab-ko-for-Google-Colab/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthr

 # 1**. 데이터 로드**


In [ ]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab
     

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")


('ratings_total.txt', <http.client.HTTPMessage at 0x7ff914425370>)

In [ ]:

total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


# **2. hgtk 튜토리얼**


In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('ㄱ')

True

In [ ]:

# 한글인지 체크
hgtk.checker.is_hangul('28')

False

In [ ]:
# 음절을 초성, 중성, 종성으로 분해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [ ]:
# 초성, 중성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

In [ ]:
# 초성, 중성, 종성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')


'남'

In [ ]:
# 한글이 아닌 입력에 대해서는 에러 발생.
hgtk.letter.decompose('1')

NotHangulException: ignored

In [ ]:

# 결합할 수 없는 상황에서는 에러 발생
hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

NotHangulException: ignored

# **3. 데이터 전처리**

In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char
    
  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [ ]:
word_to_jamo('남동생')


'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [ ]:
word_to_jamo('여동생')


'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [ ]:
mecab = Mecab()


In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))


['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [ ]:

def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))


['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [ ]:
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_list()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [01:16<00:00, 2625.87it/s]


In [ ]:
len(tokenized_data)


200000

In [ ]:
tokenized_data[0]


['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0
  
  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']
  
  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:  
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word
     

In [ ]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')


'남동생'

# **4. FastText**

In [ ]:
import fasttext


In [ ]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 271851.04 line/s]


In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')


In [ ]:
model.save_model("fasttext.bin")


In [ ]:
model = fasttext.load_model("fasttext.bin")


In [ ]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'


array([ 1.6027859e-01, -6.0529500e-01, -9.3685311e-01,  4.4752705e-01,
       -4.4802988e-01, -6.2819704e-02, -5.8167964e-02, -1.3665992e-01,
        3.3077797e-01,  4.4929687e-02,  1.9705124e-01, -4.2070866e-01,
        2.3679982e-01, -7.0590895e-01, -2.6355702e-01, -2.6564488e-01,
        7.8167218e-01,  2.5098011e-01, -6.9242799e-01, -3.9730206e-01,
       -3.9197277e-02, -4.4306219e-01, -9.4398046e-01, -5.7922357e-01,
        6.8800306e-01, -2.6219574e-01, -5.0067717e-01,  8.2207060e-01,
        9.6493536e-01, -3.6625880e-01,  6.7887002e-01, -2.9598567e-01,
        6.7255163e-01, -1.0686591e+00, -3.6052641e-01,  5.9854919e-01,
       -1.1421274e+00, -1.9017097e-01, -6.7064470e-01, -3.8263604e-01,
       -2.4509324e-01,  1.2382239e-01, -1.0761876e-01,  8.9628732e-01,
       -2.7793965e-01, -1.0207076e+00, -2.7689618e-01, -5.4685321e-02,
       -1.4878145e-01,  1.6172297e-01,  1.0738994e-01,  1.2085098e+00,
       -2.6960951e-01, -2.9162854e-01,  4.6823856e-01,  2.9045030e-01,
      

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)


[(0.8924053907394409, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8393558263778687, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7613360285758972, 'ㅊㅣㄴㄱㅜ-'),
 (0.7612759470939636, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.753082275390625, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7328363656997681, 'ㅇㅓㄴㄴㅣ-'),
 (0.7245802879333496, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.7165004014968872, 'ㄴㅏㅁㅇㅏ-'),
 (0.7118499279022217, 'ㅎㅏㄱㅅㅐㅇ'),
 (0.7089409828186035, 'ㅍㅕㅇㅅㅐㅇ')]

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))


[('동생', 0.8924053907394409), ('남친', 0.8393558263778687), ('친구', 0.7613360285758972), ('남편', 0.7612759470939636), ('생일', 0.753082275390625), ('언니', 0.7328363656997681), ('중학생', 0.7245802879333496), ('남아', 0.7165004014968872), ('학생', 0.7118499279022217), ('평생', 0.7089409828186035)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동쉥'), k=10)))


[('남동생', 0.8965290784835815), ('남친', 0.8175991773605347), ('남매', 0.7625890970230103), ('남짓', 0.7170418500900269), ('남녀', 0.7148666977882385), ('남아', 0.7067198753356934), ('남김', 0.701329231262207), ('남겼', 0.7007875442504883), ('남음', 0.6943906545639038), ('나눴', 0.6844045519828796)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동셍ㅋ'), k=10)))


[('남동생', 0.813254177570343), ('남친', 0.7305951714515686), ('남녀', 0.640708863735199), ('남김', 0.632788360118866), ('남매', 0.6295064687728882), ('남짓', 0.6270017623901367), ('남아', 0.6175503730773926), ('남길', 0.6078289747238159), ('남음', 0.6073477268218994), ('동생', 0.6022053956985474)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('난동생'), k=10)))


[('남동생', 0.8587841987609863), ('난생', 0.8366194367408752), ('동생', 0.7845795750617981), ('남편', 0.7702183127403259), ('남아', 0.7513651251792908), ('남친', 0.7411659955978394), ('중학생', 0.7367032170295715), ('고등학생', 0.7166954874992371), ('학생', 0.7054546475410461), ('나눴', 0.6840975284576416)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('낫동생'), k=10)))


[('남동생', 0.9258953928947449), ('동생', 0.8999727368354797), ('중학생', 0.7580940127372742), ('남편', 0.7509303689002991), ('남친', 0.7369559407234192), ('학생', 0.728604257106781), ('친구', 0.7280032634735107), ('난생', 0.7205984592437744), ('평생', 0.7159558534622192), ('남아', 0.6951503157615662)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('납동생'), k=10)))


[('남동생', 0.9067105650901794), ('동생', 0.8584803938865662), ('남편', 0.7900568842887878), ('남친', 0.769167423248291), ('난생', 0.7626235485076904), ('중학생', 0.7597086429595947), ('고등학생', 0.7509375810623169), ('남아', 0.7351293563842773), ('학생', 0.7301429510116577), ('초등학생', 0.7257920503616333)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('냚동생'), k=10)))


[('동생', 0.9683970212936401), ('남동생', 0.9080665707588196), ('언니', 0.7964166402816772), ('친구', 0.7869051694869995), ('조카', 0.7691419124603271), ('중학생', 0.7690635919570923), ('학생', 0.7676815390586853), ('생일', 0.7479627132415771), ('딸', 0.7355194091796875), ('평생', 0.7307422757148743)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품질'), k=10)))


[('품질', 0.8560693860054016), ('음질', 0.8343454599380493), ('땜질', 0.7461564540863037), ('찜질', 0.7019796371459961), ('퀄리티', 0.6918339729309082), ('군것질', 0.6713786721229553), ('사포질', 0.6697678565979004), ('성질', 0.6671198606491089), ('완성품', 0.647478461265564), ('고감', 0.6442110538482666)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품쥘'), k=10)))


[('고품질', 0.8332071900367737), ('재고품', 0.7801340818405151), ('재품', 0.746681272983551), ('소모품', 0.7344151139259338), ('반제품', 0.733773946762085), ('중고품', 0.7257165312767029), ('고퀄', 0.708255410194397), ('소지품', 0.7078821659088135), ('소품', 0.7056055665016174), ('제품', 0.6908734440803528)]


In [ ]:

print(transform(model.get_nearest_neighbors(word_to_jamo('노품질'), k=10)))

[('고품질', 0.9005551338195801), ('품질', 0.8693786859512329), ('음질', 0.8008126020431519), ('땜질', 0.7257964015007019), ('퀄리티', 0.7084727883338928), ('찜질', 0.6703440546989441), ('성질', 0.6589627265930176), ('화질', 0.6366837620735168), ('소모품', 0.6345481872558594), ('퀄러티', 0.6307878494262695)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('보품질'), k=10)))


[('고품질', 0.8293775320053101), ('품질', 0.7748414874076843), ('음질', 0.7540803551673889), ('보풀', 0.7300360798835754), ('땜질', 0.6797279119491577), ('사포질', 0.6518200039863586), ('바느질', 0.6513069868087769), ('찜질', 0.651176929473877), ('재질', 0.6500739455223083), ('퀄리티', 0.6445159316062927)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품'), k=10)))


[('반제품', 0.8988224267959595), ('완제품', 0.878635048866272), ('상품', 0.8517276644706726), ('타제품', 0.848635733127594), ('재품', 0.8341369032859802), ('제풍', 0.814214289188385), ('최상품', 0.7965015769004822), ('화학제품', 0.7858718633651733), ('중품', 0.7763485908508301), ('재고품', 0.7603049874305725)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품ㅋ'), k=10)))


[('제품', 0.85528165102005), ('완제품', 0.7614362835884094), ('제풍', 0.7397014498710632), ('반제품', 0.7347006797790527), ('상품', 0.7155025601387024), ('최상품', 0.711371123790741), ('타제품', 0.708276093006134), ('화학제품', 0.6600770354270935), ('완성품', 0.653258740901947), ('재품', 0.6524826884269714)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품^^'), k=10)))


[('제품', 0.9406508803367615), ('제풍', 0.8368441462516785), ('반제품', 0.8046674728393555), ('완제품', 0.7967750430107117), ('타제품', 0.7809423208236694), ('상품', 0.775849461555481), ('최상품', 0.7484093904495239), ('재품', 0.7079985737800598), ('화학제품', 0.7028535604476929), ('상품명', 0.6836664080619812)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제푼ㅋ'), k=10)))


[('제풍', 0.6739656329154968), ('제품', 0.6483213901519775), ('완제품', 0.5649111866950989), ('반제품', 0.5445183515548706), ('최상품', 0.5345538258552551), ('제왕', 0.5305442810058594), ('젝', 0.5244624018669128), ('타제품', 0.520054042339325), ('젱', 0.5110968351364136), ('상품', 0.5059307217597961)]
